In [3]:
import sympy as sp

sMul = sp.Mul

from symqups import s
from symqups.objects.base import PhaseSpaceObject, qpTypePSO, alphaTypePSO
from symqups.objects.scalars import (q, p, alpha, alphaD, W, mu, _Primed, 
                                     _DerivativeSymbol, hbar, _deprime, StateFunction)
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.operator_ordering import sOrdering, explicit, express
from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups._internal.cache import sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)

a = [alpha(i) for i in range(10)]
aop = [annihilateOp(i) for i in range(10)]
ad = [alphaD(i) for i in range(10)]
adop = [createOp(i) for i in range(10)]
x = sp.Symbol("x")
qq = [q(i) for i in range(10)]
pp = [p(i) for i in range(10)]
W = scalars.W
s.val = 0
s
scalars.mu = 1
scalars.hbar = 1

In [ ]:
test = aop[1] * adop[0]**3 *aop[0] * sp.exp(adop[1]*aop[0]) * aop[1]**2 * aop[0] 

test

\hat{a}^{\dagger}_{0}**3*\hat{a}_{0}*\hat{a}_{1}*exp(\hat{a}_{0}*\hat{a}^{\dagger}_{1})*\hat{a}_{0}*\hat{a}_{1}**2

In [10]:
def CG_transform(expr : sp.Expr) -> sp.Expr:
    """
    oper -> quantum ps vars
    """
    
    def treat_add(A : sp.Expr) -> sp.Expr:
        return sp.Add(*_mp_helper(A.args, CG_transform))
    
    def treat_mul(A : sp.Expr) -> sp.Expr:
        A_by_polynomiality = separate_term_by_polynomiality()
        
    expr = qp2a(sp.sympify(expr))